## Reveal AI final project

- *Will test multiple models in this notebook*
- resnet50
-


In [ ]:
# mount drive first
from google.colab import drive
drive.mount('/content/drive')

In [ ]:



!apt-get install -y p7zip-full
DRIVE_ZIP_PATH_1 = '/content/drive/MyDrive/Colab Notebooks/RevealAI/test.7z'
DRIVE_ZIP_PATH_2 = '/content/drive/MyDrive/Colab Notebooks/RevealAI/train.7z'
DRIVE_ZIP_PATH_3 = '/content/drive/MyDrive/Colab Notebooks/RevealAI/validation.7z'


# # Local temporary disk destination
LOCAL_DESTINATION = '/content/'

# # Execute the copy command

!cp "{DRIVE_ZIP_PATH_1}" "{LOCAL_DESTINATION}"
!cp "{DRIVE_ZIP_PATH_2}" "{LOCAL_DESTINATION}"
!cp "{DRIVE_ZIP_PATH_3}" "{LOCAL_DESTINATION}"

print(f" Copied data to local Colab disk.")
# # Path to the ZIP file on the local disk

LOCAL_ZIP_PATH_1 = '/content/test.7z'
LOCAL_ZIP_PATH_2 = '/content/train.7z'
LOCAL_ZIP_PATH_3 = '/content/validation.7z'

# # Execute the unzip command
# # -q: quiet (less terminal output)
# # -d /content/: extract contents to the /content/ directory

!7z x "{LOCAL_ZIP_PATH_1}" -o/content/
!7z x "{LOCAL_ZIP_PATH_2}" -o/content/
!7z x "{LOCAL_ZIP_PATH_3}" -o/content/
print(" Unzipping complete! Your data is now fast to access.")

In [ ]:
# !ls test

In [ ]:
!pip install timm
!pip install transformers timm ftfy

In [ ]:
#!ls

In [ ]:
# u have to get ride of the empty files or the model wont run
import os

folders = [
    "/content/train/real",
    "/content/test/ai",
    "/content/test/real",
    "/content/validation/ai",
    "/content/validation/real"
]
for folder in folders:
  for f in os.listdir(folder):
    path = os.path.join(folder, f)
    if os.path.isfile(path) and f.lower().endswith(('.jpg', '.jpeg')):
        if os.path.getsize(path) == 0:
            os.remove(path)
            print(f"Deleted empty file: {path}")

In [ ]:
# imports
import os, sys,torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import os
from PIL import Image
import torch
import torch.nn.functional as F
import timm
import torch.nn as nn
import matplotlib.pyplot as plt
import random
import torch
import torch.nn as nn
from torchvision.models import shufflenet_v2_x1_0, ShuffleNet_V2_X1_0_Weights

In [ ]:
size_check = {}
# verify that the images are the same size or not
# print(os.listdir('1_fake')[0:10])
for img in os.listdir('/content/train/ai'):
    if '.ipynb_checkpoints' == img:
        continue
    images=Image.open('/content/train/ai/'+img)
    # print(images.size)
    if images.size in size_check:
        size_check[images.size] += 1
    else:
        size_check[images.size] = 1
# sizes are not the same
print(list(size_check.items())[0:10])


In [ ]:
# Resize images for convnext
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])


# Same transforms as your training
tf_train = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

tf_eval = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

# Load datasets
train_dataset = datasets.ImageFolder('/content/train', transform=tf_train)
val_dataset   = datasets.ImageFolder('/content/validation', transform=tf_eval)
test_dataset  = datasets.ImageFolder('/content/test', transform=tf_eval)

print(f"Classes: {train_dataset.classes}")
print(f"Total images: {len(train_dataset)}")

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=128, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=128, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

## Model Convnext Tiny Model

In [ ]:

model = timm.create_model('convnext_tiny', pretrained=True, num_classes=2)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = model.to(device)

print("Model ready on device:", device)

In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
for epoch in range(10):

    # -------------------------
    # TRAINING
    # -------------------------
    model.train()
    train_total, train_correct, train_loss = 0, 0, 0

    for imgs, labels in train_loader:
        imgs, labels = imgs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(imgs)
        loss = F.cross_entropy(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        preds = outputs.argmax(1)
        train_correct += (preds == labels).sum().item()
        train_total += labels.size(0)

    train_acc = 100 * train_correct / train_total
    train_loss = train_loss / len(train_loader)


    # -------------------------
    # VALIDATION
    # -------------------------
    model.eval()
    val_total, val_correct, val_loss = 0, 0, 0

    with torch.no_grad():
        for imgs, labels in val_loader:
            imgs, labels = imgs.to(device), labels.to(device)

            outputs = model(imgs)
            loss = F.cross_entropy(outputs, labels)

            val_loss += loss.item()
            preds = outputs.argmax(1)
            val_correct += (preds == labels).sum().item()
            val_total += labels.size(0)

    val_acc = 100 * val_correct / val_total
    val_loss = val_loss / len(val_loader)

    # -------------------------
    # LOGGING
    # -------------------------
    print(f"Epoch {epoch+1}")
    print(f"  Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.2f}%")
    print(f"  Val   Loss: {val_loss:.4f} | Val   Acc: {val_acc:.2f}%")
    print("-" * 50)

In [ ]:
import random

# pick random image from test/ai
folder = "/content/test/ai"
img_name = random.choice(os.listdir(folder))
img_path = folder + "/" + img_name

# same transform as eval
tf = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

img = Image.open(img_path).convert("RGB")
x = tf(img).unsqueeze(0).to(device)

model.eval()
with torch.no_grad():
    pred = model(x).argmax(1).item()

label_names = train_dataset.classes  # ['ai', 'real']

print("Predicted class:", label_names[pred])

plt.imshow(img)
plt.title(f"Predicted: {label_names[pred]}")
plt.axis("off")
plt.show()

## Model Vision Transformer

In [ ]:

model = timm.create_model('vit_large_patch14_224', pretrained=False, num_classes=2)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = model.to(device)

print("Model ready on device:", device)

In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)

for epoch in range(10):

    # -------------------------
    # TRAINING
    # -------------------------
    model.train()
    train_total, train_correct, train_loss = 0, 0, 0

    for imgs, labels in train_loader:
        imgs, labels = imgs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(imgs)
        loss = F.cross_entropy(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        preds = outputs.argmax(1)
        train_correct += (preds == labels).sum().item()
        train_total += labels.size(0)

    train_acc = 100 * train_correct / train_total
    train_loss = train_loss / len(train_loader)


    # -------------------------
    # VALIDATION
    # -------------------------
    model.eval()
    val_total, val_correct, val_loss = 0, 0, 0

    with torch.no_grad():
        for imgs, labels in val_loader:
            imgs, labels = imgs.to(device), labels.to(device)

            outputs = model(imgs)
            loss = F.cross_entropy(outputs, labels)

            val_loss += loss.item()
            preds = outputs.argmax(1)
            val_correct += (preds == labels).sum().item()
            val_total += labels.size(0)

    val_acc = 100 * val_correct / val_total
    val_loss = val_loss / len(val_loader)

    # -------------------------
    # LOGGING
    # -------------------------
    print(f"Epoch {epoch+1}")
    print(f"  Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.2f}%")
    print(f"  Val   Loss: {val_loss:.4f} | Val   Acc: {val_acc:.2f}%")
    print("-" * 50)

In [ ]:
import random

# pick random image from test/ai
folder = "/content/test/ai"
img_name = random.choice(os.listdir(folder))
img_path = folder + "/" + img_name

# same transform as eval
tf = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

img = Image.open(img_path).convert("RGB")
x = tf(img).unsqueeze(0).to(device)

model.eval()
with torch.no_grad():
    pred = model(x).argmax(1).item()

label_names = train_dataset.classes  # ['ai', 'real']

print("Predicted class:", label_names[pred])

plt.imshow(img)
plt.title(f"Predicted: {label_names[pred]}")
plt.axis("off")
plt.show()

## Model Resnet50

In [ ]:

model = timm.create_model('resnet50', pretrained=True, num_classes=2)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = model.to(device)

print("Model ready on device:", device)

In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)

for epoch in range(10):

    # -------------------------
    # TRAINING
    # -------------------------
    model.train()
    train_total, train_correct, train_loss = 0, 0, 0

    for imgs, labels in train_loader:
        imgs, labels = imgs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(imgs)
        loss = F.cross_entropy(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        preds = outputs.argmax(1)
        train_correct += (preds == labels).sum().item()
        train_total += labels.size(0)

    train_acc = 100 * train_correct / train_total
    train_loss = train_loss / len(train_loader)


    # -------------------------
    # VALIDATION
    # -------------------------
    model.eval()
    val_total, val_correct, val_loss = 0, 0, 0

    with torch.no_grad():
        for imgs, labels in val_loader:
            imgs, labels = imgs.to(device), labels.to(device)

            outputs = model(imgs)
            loss = F.cross_entropy(outputs, labels)

            val_loss += loss.item()
            preds = outputs.argmax(1)
            val_correct += (preds == labels).sum().item()
            val_total += labels.size(0)

    val_acc = 100 * val_correct / val_total
    val_loss = val_loss / len(val_loader)

    # -------------------------
    # LOGGING
    # -------------------------
    print(f"Epoch {epoch+1}")
    print(f"  Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.2f}%")
    print(f"  Val   Loss: {val_loss:.4f} | Val   Acc: {val_acc:.2f}%")
    print("-" * 50)

In [ ]:
import random

# pick random image from test/ai
folder = "/content/test/ai"
img_name = random.choice(os.listdir(folder))
img_path = folder + "/" + img_name

# same transform as eval
tf = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

img = Image.open(img_path).convert("RGB")
x = tf(img).unsqueeze(0).to(device)

model.eval()
with torch.no_grad():
    pred = model(x).argmax(1).item()

label_names = train_dataset.classes  # ['ai', 'real']

print("Predicted class:", label_names[pred])

plt.imshow(img)
plt.title(f"Predicted: {label_names[pred]}")
plt.axis("off")
plt.show()

## Model ShuffleNetV2

In [ ]:


num_classes = 2
model = shufflenet_v2_x1_0(weights=ShuffleNet_V2_X1_0_Weights.IMAGENET1K_V1)
model.fc = nn.Linear(1024, num_classes)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = model.to(device)
print("Model ready on device:", device)

In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)

for epoch in range(10):

    # -------------------------
    # TRAINING
    # -------------------------
    model.train()
    train_total, train_correct, train_loss = 0, 0, 0

    for imgs, labels in train_loader:
        imgs, labels = imgs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(imgs)
        loss = F.cross_entropy(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        preds = outputs.argmax(1)
        train_correct += (preds == labels).sum().item()
        train_total += labels.size(0)

    train_acc = 100 * train_correct / train_total
    train_loss = train_loss / len(train_loader)


    # -------------------------
    # VALIDATION
    # -------------------------
    model.eval()
    val_total, val_correct, val_loss = 0, 0, 0

    with torch.no_grad():
        for imgs, labels in val_loader:
            imgs, labels = imgs.to(device), labels.to(device)

            outputs = model(imgs)
            loss = F.cross_entropy(outputs, labels)

            val_loss += loss.item()
            preds = outputs.argmax(1)
            val_correct += (preds == labels).sum().item()
            val_total += labels.size(0)

    val_acc = 100 * val_correct / val_total
    val_loss = val_loss / len(val_loader)

    # -------------------------
    # LOGGING
    # -------------------------
    print(f"Epoch {epoch+1}")
    print(f"  Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.2f}%")
    print(f"  Val   Loss: {val_loss:.4f} | Val   Acc: {val_acc:.2f}%")
    print("-" * 50)

In [ ]:
model.eval()
test_correct = 0
test_total = 0

with torch.no_grad():
    for imgs, labels in test_loader:
        imgs, labels = imgs.to(device), labels.to(device)
        outputs = model(imgs)
        preds = outputs.argmax(1)
        test_correct += (preds == labels).sum().item()
        test_total += labels.size(0)

print("Test Accuracy:", 100 * test_correct / test_total)

In [ ]:


# Folder to sample from
folder = "/content/test/ai"
img_name = random.choice(os.listdir(folder))
img_path = folder + "/" + img_name

# Same transform as eval
tf = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

# Load + preprocess
img = Image.open(img_path).convert("RGB")
x = tf(img).unsqueeze(0).to(device)

# Predict
model.eval()
with torch.no_grad():
    pred = model(x).argmax(1).item()

label_names = train_dataset.classes  # ['ai', 'real']

print("Random test image:", img_name)
print("Predicted class:", label_names[pred])

plt.imshow(img)
plt.title(f"Predicted: {label_names[pred]}")
plt.axis("off")
plt.show()

## CLIP VIT